In [34]:
import os
import json
from pathlib import Path

In [35]:
DATA_RAW_DIR = 'data/raw'
DATA_RAW_FILE = 'tiendapet.jl'

project_dir = Path().cwd().parent
data_raw_path = project_dir.joinpath(DATA_RAW_PATH)
data_raw_file_path = data_raw_path.joinpath(DATA_RAW_FILE)

In [46]:
j_content = []
with open(data_raw_file_path) as f:
    for line in f:
        j_content.append(json.loads(line))

## Nutritional Facts table built
Parsing the field in each json line to accomodate a table that has (product_name - nutritional facts). Columns are fixed for now
* nf_df = nutritional facts data frame

In [136]:
import pandas as pd

In [137]:
nf_df = pd.DataFrame(columns=["product_name", "protein", "fat", "fiber", "humidity", "food_ashes", "calcium", "phosphorus"])

In [302]:
def filter_nutritional_fact_lines(data: list[dict]) -> list:
    """"""
    search = re.compile(r'[a-z0-9.]').search
    filtered_lines = []
    for line in data:
        if search(''.join(line.get('nutritional_facts', ''))):
            filtered_lines.append(line)
    return filtered_lines


In [426]:
def has_any_digits(string: str) -> bool:
    """"""
    return any(str.isdigit(l) for l in string)

def filter_protein_lines(nutritional_facts_list: list) -> str:
    """"""
    search = re.compile(r'prote\wnas?\s(?:\w+\s).*?(?:\()?((?:\d+)(?:.)?(?:\d+.?%))(?:\))?|prote\wnas?:?\s(?:\()?(\d+.?%)(?:\))?|prote\wnas?\s(?:\w+\s?:?).*?((?:\d+)?(?:.)?(?:\d+.?%))|prote\wna?s?:?(?:\\\w+)?\s+(?:\.+)?((?:\d+)(?:.)?(?:\d+.?%))|prote\wna\s(?:\(\w+\)).*?(\d+.?%)', re.MULTILINE | re.IGNORECASE | re.UNICODE).search
    p = None
    for _ in nutritional_facts_list:
        result = search(_)
        if result:
            p = ''.join([_ for _ in result.groups() if _])
            break
    search = re.compile(r'\bprote\wna?s?\b(?:\s)?(?:\w+)?', re.MULTILINE | re.IGNORECASE).search
    if not p:
        for idx, _ in enumerate(nutritional_facts_list):
            result = search(_)
            if result:
                try:
                    if has_any_digits(nutritional_facts_list[idx + 3]):
                        p = nutritional_facts_list[idx + 3]
                    elif has_any_digits(nutritional_facts_list[idx + 6]):
                        p = nutritional_facts_list[idx + 6]
                    elif has_any_digits(nutritional_facts_list[idx + 4]):
                        p = nutritional_facts_list[idx + 4]
                    elif has_any_digits(nutritional_facts_list[idx + 2]):
                        p = nutritional_facts_list[idx + 2]
                    elif has_any_digits(nutritional_facts_list[idx + 1]):
                        p = nutritional_facts_list[idx + 1]
                except IndexError:
                    p = None
                break
    return p

In [469]:
def protein_value_helper(protein_list:list) -> None:
    """manual helper for protein list """
    l = protein_list
    l[259] = '26%'
    l[265] = '23%'
    l[431] = '18%'
    return l
    

protein_list = []
for idx, line in enumerate(nf_lines):
    protein = filter_protein_lines(line["nutritional_facts"])
    protein_list.append(protein)

product_names = [i['product_name'] for i in nf_lines]
protein_list = protein_value_helper(protein_list)

In [471]:
nf_df["product_name"] = product_names
nf_df["protein"] = protein_list

In [472]:
nf_df

,product_name,protein,fat,fiber,humidity,food_ashes,calcium,phosphorus
0,Cachorro Pollo 15Kg Cachorro Pollo 15Kg,29%,NaN,NaN,NaN,NaN,NaN,NaN
1,Dogxtreme Adult Senior Light Dogxtreme Adult S...,20 %,NaN,NaN,NaN,NaN,NaN,NaN
2,Dogxtreme Puppy Chicken With Rice Dogxtreme Pu...,30 %,NaN,NaN,NaN,NaN,NaN,NaN
3,Dogxtreme Adult Salmon & Rice Dogxtreme Adult ...,None,NaN,NaN,NaN,NaN,NaN,NaN
4,Adulto Pollo 15Kg Adulto Pollo 15Kg,25%,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
580,Dog Chow Abrazzos Integral Mini Dog Chow Abraz...,"9,0%",NaN,NaN,NaN,NaN,NaN,NaN
581,Dog Chow Pouch Cordero Dog Chow Pouch Cordero,"10,0%",NaN,NaN,NaN,NaN,NaN,NaN
582,Dog Chow Pouch Raza Pequeña Festival De Salmon...,"10,5%",NaN,NaN,NaN,NaN,NaN,NaN
583,Dog Chow Pouch Raza Pequeña Trozos Jugosos De ...,11%,NaN,NaN,NaN,NaN,NaN,NaN
